Previously: 

- talked about "sweet spot" in Nora's 2_BTP model compared to David's 
    - plotted 20 versions of Nora's 2_BTP model, from 120-140 species
    - found that the addition of the 130th species "fixed" the speed
    - adding in the 133rd species (and beyond) made it "worse" again 
- confirmed that correct NIST model was used. Used David's script and got similar flame speeds using NIST model. 
    - using both my script and David's, can't exactly recreate the NIST flame speeds across various 2-BTP mole fractions
        - SANDIA PREMIX code v. Cantera? 
    

/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/NK_120_to_140/plots/plot_speeds_120-140

# Investigating Nora's 2_BTP Model

## 129 - 130 species : "Fixes" flamespeed

Initial findings: 

- 130 model (1): has 1313 reactions

- 129 model (2): has 1269 reactions

- difference between the models: 
    
    - from 129 to 130 model, The species added in was: C3H3(6822)

     - difference of 44 reactions, only 10 of which is pressure dependent

    - of the 44 new reactions, 40 involve the new species.
    
(/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/NK_120_to_140/scripts)

    

**objective**: starting with the smaller model (129 species), let's add in the 44 reactions one at a time and see if there is a species reaction that "fixes" the flamespeed. Basically, expand the smaller model one reaction at a time so it becomes the larger model (130 species).

(/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/NK_120_to_140/plot_one_flip_models)

- plots show that rxn with index # 1287 "fixes" the flamespeed:

  C#CC(5272) <=> H(8) + C3H3(6822)
      - using .is_forward, found out that this reaction does not proceed in the forward direction. 
      - reaction consumes hydrogen radicals and involves C3H3.  

# 132-133 species: "Screws Up" flamespeed 

Initial findings: 

- 133 model (1): has 1326 rxns

- 132 model (2): has 1321 reactions 


    - from 132 to 133 model, the species added in was: S(7874)
    - difference of 5 reactions, only 1 of which is pressure dependent
        - of the 5 reactions, only 1 involves the new species. 
        - 3 of the 5 involve reactions that PRODUCE a hydrogen radical. opposite of last experiment. 
            - .is_forward = False for each of these reactions
            - Also involves C3H3, the species that was added in from 129 - 130 
        - S(4579) <=> S(7874)
        - H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23)
        - H(8) + C3H3(6822) <=> CH3(19) + C2H(22)
        - H(8) + C3H3(6822) <=> CH2(S)(25) + H2CC(24)
        - CF2O2(848) + C2H5(32) <=> S(2391) + C2H4(30)

- from plots: 
    - 1322 (same as smaller model) looks "correct"
    - adding in rxn #1323 makes it"worse" : H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23) 
    - adding in rxn #1323 makes it a little more "worse" : H(8) + C3H3(6822) <=> CH3(19) + C2H(22)




## Conclusion 

from : https://www.nature.com/articles/s41598-022-08648-5.pdf?origin=ppub

"The production of OH radicals is immensely dependent upon the availability of H radicals. Hence, the reactions consuming H radicals like the H-abstraction of the fuel, CH4+H ⇔ CH3+ H2, compete with the OH forming reactions and therefore has a large negative sensitivity. Also, this reaction is a chain propagation reaction as opposed to the chain branching reaction: O2+H ⇔ O+OH, which produces two reactive radicals that can help initiate and sustain the flame. Similarly, the H producing reactions for, e.g., 2CH3 ⇔ H+ C2H5 have a positive sensitivity. Correspondingly, the OH producing reactions such as CH3+ O2 ⇔ CH2O+OH, CH2O+ O2 ⇔ HCO+OH, and CH3+ HO2 ⇔ CH3O+OH have a positive impact upon OH availability. All these reactions are again chain branching reactions. Te reaction 2CH3 (+M) ⇔ C2H6 (+M), competes with the reaction between CH3 with O2 and HO2 to produce OH radicals and thus has a negative sensitivity"

Next question: how significant is this reaction? is it consuming/producing a significant amount of H radicals for this reasoning to be true?

- From the following experiments, we know that there are several important reactions to focus on in Nora's model
    - start with "bad" model, with 129 species added in core
    - at 130 species, add in C#CC(5272) <=> H(8) + C3H3(6822) which "fixes" speed
    - at 133 species, several reactions added in to make it worse again:
            - H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23)
            - H(8) + C3H3(6822) <=> CH3(19) + C2H(22)



These equations above ARE in the NIST model, too. 


- In the different models
    - (NIST).......(RMG).......... Equation
    -  285 ........ 1287............. H(8) + C3H3(6822) <=> C#CC(5272) ..............................| C3H3 + H <=> pC3H4    
    -  164 ........ 1321............. H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23) .............|  C2H2 + CH2 <=> C3H3 + H
    -  107 ........ 1322............. H(8) + C3H3(6822) <=> CH3(19) + C2H(22)................... |   C2H + CH3 <=> C3H3 + H


NEXT STEP:

- Maybe RMG has "bad" kinetics for these reactions (specifically the last 2). 
- Recreate Nora's RMG model of 2-BTP, replacing some or all of these 3 reactions with NIST kinetics.

- ran 8 models, each with the following reaction kinetics changed to NIST: 

    - [ ]
    - [1321]
    - [1322]
    - [1285]
    - [1321, 1322]
    - [1321, 1285]
    - [1322, 1285]
    - [1321, 1322, 1285]

Results: 

- Failed utterly. Confirmed that these kinetics were changed, but negligable effect on flame speed. 

Thoughts: 

- Since these reactions exist in both models, double check that they are in the same direction. 
- Is NIST an optimized model? 
- Maybe there's more reactions in NIST that consume radical H's, so although they included the last two reactions, there are more "consuming H" reactions to balance out the production of H? 